In [1]:
from os import path, mkdir, makedirs
from os import makedirs

import numpy as np
import pandas as pd

from pyts.image import GramianAngularField
import matplotlib.pyplot as plt
from tqdm import tqdm

Filepaths

In [2]:
dataset_dir = path.join('..', '..', 'Data', 'UCI HAR Dataset')
hparams_dir = path.join('..', 'hyperparameters')

## Reading and transforming time series

In [3]:
sensor_types = ['body_acc', 'body_gyro', 'total_acc']
axis = ['x', 'y', 'z']

In [4]:
def get_dataset(col_type, dataset_type, start=0, stop=0):
    if col_type == 'X':
        horizontal_images = list()
        dataset = list()
        for sensor in sensor_types:
            for direction in axis:
                filepath = path.join(dataset_dir, dataset_type, 'Inertial Signals', '_'.join([sensor,direction,dataset_type])+'.txt')
                gaf = GramianAngularField(method="difference")
                data = pd.read_csv(filepath, delim_whitespace=True, header=None)
                gaf.fit(data)                
                dataset.append(gaf.transform(data[start:stop]))
        return dataset
    else:
        return pd.read_csv(path.join(dataset_dir, dataset_type, 'y_' + dataset_type + '.txt'), header=None)

In [5]:
classes = {1:'WALKING', 2:'WALKING_UPSTAIRS', 3:'WALKING_DOWNSTAIRS', 4:'SITTING', 5:'STANDING', 6:'LAYING'}

In [6]:
y_train = get_dataset('y', 'train')

In [9]:
y_test = get_dataset('y', 'test')

## Saving Images

In [10]:
def save_images_by_class(dataset_type, y):
    prev = 0
    count = 1
    if dataset_type == 'train':
        start = 919
        stop = 7353
        step = 919
    else:
        start = 1474
        stop = 2949
        step = 1474
    for i in range(start, stop, step):
        X = get_dataset('X', dataset_type, prev, i)
        for j, item in enumerate(X):
            count = prev+1
            for k in tqdm(range(len(item))):
                makedirs(path.join('..', '..', 'Data', 'toBeStackedImagesGADF', dataset_type, classes[y[0][count-1]], str(count)), exist_ok=True)
                plt.imsave(path.join('..', '..', 'Data', 'toBeStackedImagesGADF', dataset_type, classes[y[0][count-1]], str(count), str(j)+'.png'), item[k], cmap="jet")
                count += 1
        prev = i


In [11]:
save_images_by_class('test', y_test)

100%|██████████| 1473/1473 [00:13<00:00, 107.82it/s]


In [ ]:
save_images_by_class('train', y_train)